In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from os.path import abspath, join, isfile
from os import remove
from shutil import rmtree
from glob import glob

from pyiron_atomistics import Project
from pyiron_contrib.atomistics.mean_field.core.bond_analysis import StaticBondAnalysis

def cleanup_job(job):
    """
    Removes all the child jobs (files AND folders) to save disk space and reduce file count, and only keeps
    the hdf file.
    """
    for f in glob(abspath(join(job.working_directory, '../..')) + '/' + job.job_name + '_*'):
        if isfile(f):
            remove(f)
        else:
            rmtree(f)

In [ ]:
alpha=1.5
pr = Project('morse_al/md_runs_nvt/alpha_' + str(alpha).replace('.', '_'))
# pr.remove_jobs(recursive=True)

In [ ]:
# potential functions
D = 0.1
a_0 = 2.856
kappa = 0.

# for lammps
def md_morse(D=D, alpha=alpha, r_0=a_0, b=1):
    config = 'atom_style bond\nbond_style morse\n'
    for i in range(b):
        vals = (i+1, D, alpha, a_0)
        config += 'bond_coeff %d %.7f %.7f %.7f\n'%(vals)
    return pd.DataFrame({'Name': ['Morse'],
                         'Filename': [[]],                             
                         'Model'   : ['Morse'], 
                         'Species' : [['Al']], 
                         'Config'  : [[config]]})

In [ ]:
# standard stuff

element = 'Al'
supercell_size = 4
n_atoms = 4*supercell_size**3
samples = 5
md_steps = 1e4
md_samples = md_steps / 2000
temperatures = np.linspace(100, 900, 9)
base_structure = pr.create.structure.bulk(name=element, cubic=True).repeat(supercell_size)

In [ ]:
# relax the structure to atm pressure

minim_job = pr.create.job.Lammps('minim_job', delete_existing_job=True)
minim_job.structure = base_structure
minim_job.potential = md_morse()
minim_job.calc_minimize(pressure=0.0001)
minim_job.run()
structure = minim_job.get_structure()
a_0 = (structure.cell/supercell_size/np.sqrt(2))[0][0]
U_0 = minim_job.output.energy_pot[-1]/n_atoms

In [ ]:
# analyze bonds and get rotations and displacement matrix

stat_ba = pr.create_job(StaticBondAnalysis, 'stat_ba', delete_existing_job=True)
stat_ba.input.structure = structure.copy()
stat_ba.input.n_shells = 1
stat_ba.run()

In [ ]:
# from the static bond analysis, create a bonds list that can be passed to a pyiron lammps job

def get_bonds_list(bond_relations):
    # for FCC, only include 6 bonds out of 12, as other 6 are anti-parallel
    bonds_list = bond_relations[::2]
    for (per_bond_relations, i) in zip(bonds_list, np.arange(len(bonds_list))+1):
        # change bond type index
        per_bond_relations[:, 2] = i
    return bonds_list.reshape(-1, 3)

In [ ]:
## Run NVT job first and then run NVE

In [ ]:
## NVT

# for i, temp in enumerate(temperatures):
#     temp_group = pr.create_group('temp_' + str(i))
#     for j in range(samples):
#         job = temp_group.create.job.Lammps('npt_temp_' + str(i) + '_sample_' + str(j), delete_existing_job=True)
#         job.structure = structure.copy()
#         job.structure.bonds = get_bonds_list(stat_ba.output.per_shell_bond_relations[0].copy())
#         job.potential = md_morse(b=6)
#         job.calc_md(temperature=temp, pressure=None, n_ionic_steps=md_steps, n_print=md_samples,
#                     langevin=True, pressure_damping_timescale=100., time_step=1.)
#         job.input.control.energy_pot_per_atom()
#         job.write_restart_file()
#         job.server.queue = 'cmti'
#         job.server.cores = 4
#         job.server.runtime = 3600
#         job.run()

In [ ]:
## NVE

for i, temp in enumerate(temperatures):
    temp_group = pr.create_group('temp_' + str(i))
    for j in range(samples):
        job = pr.load('npt_temp_' + str(i) + '_sample_' + str(j))
        job_nve = job.restart(job_type=pr.job_type.Lammps, job_name='nve_temp_' + str(i) + '_sample_' + str(j))
        job_nve.calc_md(temperature=None, n_print=md_samples, n_ionic_steps=md_steps)
        del job_nve.input.control["fix___langevin"]
        job.server.queue = 'cmti'
        job.server.cores = 4
        job.server.runtime = 3600
        job_nve.run()

In [ ]:
## to delete all LAMMPS files, and only keep the pyiron job

for i in range(len(temperatures)):
    for j in range(samples):
        cleanup_job(job_npt)
        cleanup_job(job_nve)